## Установка нужных версий библиотек

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1oSFOP0j25OZAuhD8YXxyQXNTdr2lUdtn' -O requirements.txt

--2021-11-12 16:35:07--  https://drive.google.com/uc?export=download&id=1oSFOP0j25OZAuhD8YXxyQXNTdr2lUdtn
Resolving drive.google.com (drive.google.com)... 108.177.111.113, 108.177.111.138, 108.177.111.102, ...
Connecting to drive.google.com (drive.google.com)|108.177.111.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i3q46tli4nqtr7sttbgok9smmr03o0pm/1636734900000/14904333240138417226/*/1oSFOP0j25OZAuhD8YXxyQXNTdr2lUdtn?e=download [following]
--2021-11-12 16:35:07--  https://doc-0g-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i3q46tli4nqtr7sttbgok9smmr03o0pm/1636734900000/14904333240138417226/*/1oSFOP0j25OZAuhD8YXxyQXNTdr2lUdtn?e=download
Resolving doc-0g-c0-docs.googleusercontent.com (doc-0g-c0-docs.googleusercontent.com)... 142.250.103.132, 2607:f8b0:4001:c2b::84
Connecting to doc-0g-c0-docs.googleusercontent.com (d

In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np

Метрики в машинном обучении нужны для того, чтобы оценивать качество работы модели, чтобы понимать, а насколько модель хорошо запомнила закономерности из обучающей выборки.  

Сегодня будем рассматривать метрики для задачи регрессии - предсказания непрерывной целевой переменной. Для такой задачи хочется, чтобы наши модели предсказывали числа как можно ближе к истинным, чтобы отклонения реального значения и прогноза модели были минимальными.

<img src='https://miro.medium.com/max/864/0*OVG0UBquPwSiwiUi' width=600>


Для подсчета разных метрик регрессии возьмем данные из стандартных датасетов в `sklearn`.

## Получение данных

Будем работать с набором данных для задачи регрессии (целевая переменная - стоимость дома) `load_boston`, который можно получить из стандартных датасетов в `sklearn'e`.

После `load_boston()` возвращается словарь с данными (`data`), целевой переменной (`target`), названиями характеристик в данных (`feature_names`) и описанием данных (`DESCR`).

In [ ]:
from sklearn.datasets import load_boston

data = load_boston()
data.keys()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])

In [ ]:
#data.to_excel('c:\teams.xlsx')



AttributeError: ignored

In [ ]:
print(data.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [ ]:
X = data.data
features = data.feature_names
y = data.target

**Из** признаков (характеристик данных) и целевой переменной сформируем датафрейм, в качестве названий колонок возьмем названия признаков.

In [ ]:
df = pd.DataFrame(X, columns=features)
df['target'] = y

df.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9


In [ ]:
#data.to_csv('teams.xlsx')

df.to_excel("boston.xlsx")


## Обучение модели

Возьмем линейную модель из `sklearn'a` и обучим на выборке.



In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Теперь сделаем предсказания на всей выборке, чтобы понять, насколько хорошо модель научилась предсказывать стоимость дома.

In [ ]:
pred = lr.predict(X)

In [ ]:
lr = LinearRegression()
lr.fit(X,y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
#pred = lr.predict(X)
#pred_df = pd.DataFrame({'a':y,
                        'b':pred})
#pred_df

,a,b
0,24.0,30.003843
1,21.6,25.025562
2,34.7,30.567597
3,33.4,28.607036
4,36.2,27.943524
...,...,...
501,22.4,23.533341
502,20.6,22.375719
503,23.9,27.627426
504,22.0,26.127967


## Метрики

Выведем рядом истинные значения целевой переменной и предсказанные моделью значения стоимости дома.

In [ ]:
pred_df = pd.DataFrame({
    'ground truth': y,
    'prediction': pred
})

pred_df

,ground truth,prediction
0,24.0,30.003843
1,21.6,25.025562
2,34.7,30.567597
3,33.4,28.607036
4,36.2,27.943524
...,...,...
501,22.4,23.533341
502,20.6,22.375719
503,23.9,27.627426
504,22.0,26.127967


В первой строке должны были предсказать 24.0, а предсказали 30.003843, во второй строке истина 21.6, а модель сказала 25.025562, мы можем пройтись по каждому объекту и посмотреть, насколько близким оказался прогноз.

In [ ]:
pred_df['difference'] = pred_df['prediction'] - pred_df['ground truth']
pred_df

,ground truth,prediction,difference
0,24.0,30.003843,6.003843
1,21.6,25.025562,3.425562
2,34.7,30.567597,-4.132403
3,33.4,28.607036,-4.792964
4,36.2,27.943524,-8.256476
...,...,...,...
501,22.4,23.533341,1.133341
502,20.6,22.375719,1.775719
503,23.9,27.627426,3.727426
504,22.0,26.127967,4.127967


Теперь в новом столбце есть разница между предсказанием и истиной. Можем в среднем узнать, насколько эта разница большая: просуммировать отклонения и поделить на количество строк, то есть на 506.

In [ ]:
pred_df['difference'].sum() / pred_df.shape[0]

-3.3701631735656924e-16

Получилось что-то очень маленькое, максимально приближенное к нулю, значит, что в среднем наши предсказания очень близки к истине, но это не так, ведь выше наблюдали, что предскания отклоняются от истины на приличные значения.  
Усредненные отклонения в среднем дали такую маленькую ошибку из-за того, что разница может быть как положительной (если мы предсказали больше), так и отрицательной (если мы предсказали меньше), а когда мы суммируем положительные и отрицательные значения, они друг друга нивелируют, и по итогу получаем маленькую ошибку.

Получается, что нужно избавиться от знака минуса в наших предсказаниях.

### MAE

 Можем брать абсолютные значения, то есть брать модуль числа:

In [ ]:
pred_df['abs'] = abs(pred_df['prediction'] - pred_df['ground truth'])
pred_df

,ground truth,prediction,difference,abs
0,24.0,30.003843,6.003843,6.003843
1,21.6,25.025562,3.425562,3.425562
2,34.7,30.567597,-4.132403,4.132403
3,33.4,28.607036,-4.792964,4.792964
4,36.2,27.943524,-8.256476,8.256476
...,...,...,...,...
501,22.4,23.533341,1.133341,1.133341
502,20.6,22.375719,1.775719,1.775719
503,23.9,27.627426,3.727426,3.727426
504,22.0,26.127967,4.127967,4.127967


Теперь не будет такого, что отрицательные и положительные отклонения схлапываются в ноль.   
Найдем среднее отклонение по модулю для всей выборки:

In [ ]:
pred_df['abs'].mean()

3.2708628109003195

Получается мы в среднем ошибаемся на 3.27 единиц по всей выборке. Это мы подсчитали метрику под названием MAE (mean absolute error, средне-абсолютная ошибка).

$$MAE = \frac{1}{n}\sum_i^n{|y - y_{pred}|}$$

Она реализована в библиотеки `sklearn` в модуле `metrics`:


In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score



In [ ]:
#mean_absolute_error(pred_df['groundtruth'], pred_df['prediction'])

mean_absolute_error(pred_df['ground truth'], pred_df['prediction'])





KeyError: ignored

Можем не вручную считать MAE, а с помощью функции `mean_absolute_error`:

In [ ]:
mean_absolute_error(pred_df['ground truth'], pred_df['prediction'])

KeyError: ignored

Получается такое же значения метрики, как мы получали при ручном подсчете.   
Чем метрика MAE ближе к нулю, тем лучше, в этом случае мы попадаем во все истинные значения.

### MSE

Как ещё мы могли бы избавиться от знака минуса в отклонениях?  
Мы могли бы возводить отклонения в квадрат, а затем так же искать среднее по выборке:

In [ ]:
pred_df['squared'] = (pred_df['prediction'] - pred_df['ground truth']) ** 2
pred_df

,ground truth,prediction,difference,abs,squared
0,24.0,30.003843,6.003843,6.003843,36.046135
1,21.6,25.025562,3.425562,3.425562,11.734478
2,34.7,30.567597,-4.132403,4.132403,17.076757
3,33.4,28.607036,-4.792964,4.792964,22.972499
4,36.2,27.943524,-8.256476,8.256476,68.169392
...,...,...,...,...,...
501,22.4,23.533341,1.133341,1.133341,1.284461
502,20.6,22.375719,1.775719,1.775719,3.153178
503,23.9,27.627426,3.727426,3.727426,13.893705
504,22.0,26.127967,4.127967,4.127967,17.040110


Снова не будет отрицательных отклонений, которые ведут к очень маленькому усреднению.   
Найдем среднее отклонение в квадрате для всей выборки:

In [ ]:
pred_df['squared'].mean()

21.894831181729213

Получается мы в среднем ошибаемся на 21.89 квадратичных единиц по всей выборке. Эта метрика называется MSE (mean squared error, средне-квадратичная ошибка).

$$MSE = \frac{1}{n}\sum_i^n{(y - y_{pred})^2}$$

Она тоже реализована в библиотеки `sklearn` в модуле `metrics`:


In [ ]:
from sklearn.metrics import mean_squared_error

Посчитаем MSE с помощью функции `mean_squared_error`:

In [ ]:
mean_squared_error(pred_df['ground truth'], pred_df['prediction'])

21.894831181729202

И снова наши значения совпали.   
Чем метрика MSE ближе к нулю, тем лучше.

Метрика очень классная, она используется под капотом моделей машиного обучения, как функция потерь, но когда хотим понять, насколько модель ошибается, метрика MSE может вносить неясность. 

### RMSE

Отклонения считали в квадрате, поэтому если наша стоимость изменяется в рублях, то mse дает цифру рубли в квадрате, что усложняет интерпретацию результата, это легко исправить взятием корня квадратного из значения MSE - а это и есть новая метрика RMSE.

Для подсчета этой метрики можно усреднить квадратичные отклонения (то есть получить MSE), а затем взять корень квадратный:

In [ ]:
np.sqrt(pred_df['squared'].mean())

4.679191295697282

Теперь очевидно, что в среднем ошибаемся на 4.67 единиц по всей выборке, либо в большую, либо в меньшую сторону. Эта метрика RMSE (root mean squared error).

$$RMSE = \sqrt{\frac{1}{n}\sum_i^n{(y - y_{pred})^2}}$$

Она не реализована в библиотеки `sklearn`, но как мы поняли, метрика - это MSE  с корнем:


In [ ]:
np.sqrt(mean_squared_error(pred_df['ground truth'], pred_df['prediction']))

4.679191295697281

Значения идентичны.
И так же, как предыдущие метрики, чем метрика RMSE ближе к нулю, тем лучше.

### $R^2$

Первые три метрики, которые рассмотрели, имеют идеальное значение 0, к которому нужно стремиться, но эти метрики не имеют пороговых значений, выше которых мы считаем, что модель ужасная, т.к. что они в зависимости от задач могут очень сильно варьироваться.

К примеру, если наша целевая переменная изменяется от 0 до 10:

In [ ]:
y_true = [0, 3, 6, 2, 9]

Наша модель может предсказать:

In [ ]:
y_pred = [1, 5, 9, 6, 14]

Соединим предсказания и истину вместе и найдем разницу между ними:

In [ ]:
df = pd.DataFrame({
    'y_true': y_true,
    'y_pred': y_pred
})

df['difference'] = df['y_pred'] - df['y_true']
df

,y_true,y_pred,difference
0,0,1,1
1,3,5,2
2,6,9,3
3,2,6,4
4,9,14,5


И мы получим следующее значение средне-квадратичной ошибки:

In [ ]:
mean_squared_error(y_true, y_pred)

11.0

А если наша целевая переменная изменяется от 0 до 10000:

In [ ]:
y_true = [0, 300, 6000, 2000, 9900]

Наша модель может предсказать:

In [ ]:
y_pred = [1, 302, 6003, 2004, 9905]

Соединим предсказания и истину вместе и найдем разницу между ними:

In [ ]:
df = pd.DataFrame({
    'y_true': y_true,
    'y_pred': y_pred
})

df['difference'] = df['y_pred'] - df['y_true']
df

,y_true,y_pred,difference
0,0,1,1
1,300,302,2
2,6000,6003,3
3,2000,2004,4
4,9900,9905,5


И мы получим следующее значение средне-квадратичной ошибки:

In [ ]:
mean_squared_error(y_true, y_pred)

11.0

MSE получились одинаковыми, потому что одинаковые разницы в предсказаниях, но вторая модель очевидно делает меньше ошибок, чем первая, значит она более качественная, хотя значения метрики MSE абсолютно одинаковые.

MSE не позволяет сама по себе сделать вывод о том, как хорошо модель решает задачу. Вот как в нашем примере, если целевая переменная принимает значения от 0 до 10, а MSE равняется 11, это плохой показатель, а когда целевая переменная варьируется от 0 до 10000, такое же значение уже является очень хорошим.  
Для избавления от такой неясности и был введен коэффициент детерминации, который по сути является нормированной среднеквадратичной ошибкой и принимает значения от 0 до 1.


$$R^2 = 1 - \frac{\frac{1}{n}\sum^{n}_{i}{(y - y_{pred})^2}}{\frac{1}{n}\sum^{n}_{i}{(y - \bar{y})^2}}$$

где $\bar{y}=\frac{1}{l}\sum^{n}_{i}y_{i}$ - среднее значение целевой переменной.

Коэффициент детерминации характеризует, какую долю дисперсии ответов объясняет модель. Если $R^{2}=1$, то модель идеально описывает данные, если же $R^{2}$ близко к нулю, то предсказания сопоставимы по качеству с константной моделью.  
То есть, можем относиться к метрике $R^{2}$ как к точности работы модели для задачи регрессии.

Для подсчета этой метрики в числителе используется MSE нашей модели, а в знаменателе дисперсия целевой переменной (хотя к этому значению можно относиться, как к MSE не самой умной константной модели, которая везде отвечает среднее по выборке):

In [ ]:
pred_df['constant'] = pred_df['ground truth'].mean()
pred_df

,ground truth,prediction,difference,abs,squared,constant
0,24.0,30.003843,6.003843,6.003843,36.046135,22.532806
1,21.6,25.025562,3.425562,3.425562,11.734478,22.532806
2,34.7,30.567597,-4.132403,4.132403,17.076757,22.532806
3,33.4,28.607036,-4.792964,4.792964,22.972499,22.532806
4,36.2,27.943524,-8.256476,8.256476,68.169392,22.532806
...,...,...,...,...,...,...
501,22.4,23.533341,1.133341,1.133341,1.284461,22.532806
502,20.6,22.375719,1.775719,1.775719,3.153178,22.532806
503,23.9,27.627426,3.727426,3.727426,13.893705,22.532806
504,22.0,26.127967,4.127967,4.127967,17.040110,22.532806


Получаем, что MSE константной модели, которая везде отвечает среднее по выборке равно

In [ ]:
mse_const = mean_squared_error(pred_df['ground truth'], pred_df['constant'])
mse_const

84.41955615616554

А MSE нашей модели получается следующим

In [ ]:
mse_model = mean_squared_error(pred_df['ground truth'], pred_df['prediction'])
mse_model

21.894831181729202

Что лучше, чем у константных ответов, значит, наша модели описывает дисперсию целевой переменной, значит, что она уловила больше вариативности в данных, нежели простое среднее по выборке.   
Теперь можем посчитать коэффициент детерминации: поделим MSE нашей модели на дисперсию целевой переменной, а затем это значение вычтем из 1:

In [ ]:
1 - mse_model / mse_const

0.7406426641094094

$R^2$ реализована в библиотеке `sklearn`:

In [ ]:
from sklearn.metrics import r2_score

r2_score(pred_df['ground truth'], pred_df['prediction'])

0.7406426641094095

Получается $R^2$ нашей модели стремится к 1, что говорит о довольно неплохом качестве работы модели.

#### Отрицательный $R^2$

Иногда случается так, что модель получается очень плохой, что её ответы никак не соотносятся с истинными, в этом случае коэффициент детерминации может быть отрицательным.


In [ ]:
y_true = [0, 1, 2, 3, 4]
y_pred = [10, -20, -30, 40, -50]


df = pd.DataFrame({
    'y_true': y_true,
    'y_pred': y_pred
})


df['difference'] = df['y_pred'] - df['y_true']
df

,y_true,y_pred,difference
0,0,10,10
1,1,-20,-21
2,2,-30,-32
3,3,40,37
4,4,-50,-54


In [ ]:
variance = ((df['y_true'] - df['y_true'].mean()) ** 2).mean()
print(f'Дисперсия выборки {variance}')

Дисперсия выборки 2.0


In [ ]:
mse = ((df['y_true'] - df['y_pred']) ** 2).mean()
print(f'MSE {mse}')

MSE 1170.0


Полученная MSE гораздо больше дисперсии выборки, из-за этого и получается большое число при делении MSE на MSE константной модели, которая везде отвечает среднее по выборке.

In [ ]:
mse / variance

585.0

Вычитая полученное число из 1 и получается отрицательное значение:

In [ ]:
1 - mse / variance

-584.0

Или же сразу могли посчитать метрику $R^2$ через sklearn.

In [ ]:
r2_score(y_true, y_pred)

-584.0

Пугаться этого не стоит, а нужно исправить модель, возможно модель переобучилась, либо при предсказывании объектов была использована неправильная предобработка данных.

## Summary

<table>

<tr>
<td>
Метрика
</td>

<td>
Формула
</td>

<td>
Диапазоны значений
</td>

<td>
Идеальное значение
</td>
</tr>

<tr>
<td>
MAE (mean absolute error, средне-абсолютная ошибка)
</td>

<td>
$$MAE = \frac{1}{n}\sum_i^n{|y - y_{pred}|}$$

</td>

<td>
[0, +$\infty$)
</td>

<td>
0
</td>
</tr>

<tr>
<td>
MSE (mean squared error, средне-квадратичная ошибка)

</td>

<td>
$$MSE = \frac{1}{n}\sum_i^n{(y - y_{pred})^2}$$

</td>

<td>
[0, +$\infty$)
</td>

<td>
0
</td>
</tr>

<tr>
<td>
RMSE (root mean squared error).

</td>

<td>
$$RMSE = \sqrt{\frac{1}{n}\sum_i^n{(y - y_{pred})^2}}$$

</td>

<td>
[0, +$\infty$)
</td>

<td>
0
</td>
</tr>

<tr>
<td>
Коэффициент детерминации $R^{2}$

</td>

<td>
$$R^2 = 1 - \frac{\frac{1}{n}\sum^{n}_{i}{(y - y_{pred})^2}}{\frac{1}{n}\sum^{n}_{i}{(y - \bar{y})^2}}$$

</td>

<td>
(-$\infty$, 1]

</td>

<td>
1
</td>
</tr>
</table>

Метрики используются для того, чтобы получить цифру оценки работы обученной модели, чтобы уже заранее знать, чего ожидать от модели, когда ей будем давать реальные данные, а так же метрики мы можем использовать для обнаружения переобучения/недообучения. Про них разговор ведется в этом [ноутбуке](https://colab.research.google.com/drive/1MNpLMZGJCBwdNjY_PgJIKHtll6zvUEyU?usp=sharing) и в этом видео.

Муррр ♥